In [1]:
# Importing pyspiel and showing the list of supported games.
import pyspiel
print(pyspiel.registered_names())

['2048', 'airline_seats', 'amazons', 'backgammon', 'bargaining', 'battleship', 'blackjack', 'blotto', 'breakthrough', 'bridge', 'bridge_uncontested_bidding', 'catch', 'checkers', 'chess', 'cliff_walking', 'clobber', 'coin_game', 'colored_trails', 'connect_four', 'coop_box_pushing', 'coop_to_1p', 'coordinated_mp', 'cursor_go', 'dark_chess', 'dark_hex', 'dark_hex_ir', 'deep_sea', 'dou_dizhu', 'efg_game', 'euchre', 'first_sealed_auction', 'gin_rummy', 'go', 'goofspiel', 'havannah', 'hearts', 'hex', 'kriegspiel', 'kuhn_poker', 'laser_tag', 'leduc_poker', 'lewis_signaling', 'liars_dice', 'liars_dice_ir', 'maedn', 'mancala', 'markov_soccer', 'matching_pennies_3p', 'matrix_bos', 'matrix_brps', 'matrix_cd', 'matrix_coordination', 'matrix_mp', 'matrix_pd', 'matrix_rps', 'matrix_rpsw', 'matrix_sh', 'matrix_shapleys_game', 'mfg_crowd_modelling', 'mfg_crowd_modelling_2d', 'mfg_dynamic_routing', 'mfg_garnet', 'misere', 'morpion_solitaire', 'negotiation', 'nfg_game', 'nim', 'normal_form_extensive_ga

In [2]:
# Loading a game (with no/default parameters).
game = pyspiel.load_game("airline_seats")
print(game)

airline_seats()


In [3]:
# Some properties of the games.
print(game.num_players())
print(game.max_utility())
print(game.min_utility())
print(game.num_distinct_actions())

2
2.0
-2.0
2


In [6]:
# Creating initial states.
state = game.new_initial_state()
print(state)

In [7]:
# Basic information about states.
print(state.current_player())
print(state.is_terminal())
print(state.returns())
print(state.legal_actions())

-1
False
[0.0, 0.0]
[0, 1, 2]
